# Run Confluence on an HPC

# Requirements
* docker installed somewhere where you have sudo priveledges to the point where "docker --version" completes successfully
* singularity or apptainer installed on your HPC
* a dockerhub account (free)


# Overall Tasks
* Git clone all of the repos you want to run to a machine where you have sudo priveledges and where "docker --version" works (locally)
* Prep an empty_mnt directory to store confluence run (requires gdown package in environment)
* Run the "Prepare Images Locally" section of this notebook locally
* Run the "Confluence Module SLURM Script Generator" section of this notebook on your HPC to create SLURM submission scripts for each module
* Run the Confluence Driver Script Generator section of this notebook on your HPC to create a SLURM submission script that runs each of the modules one by one (the one click run)

---
## Functions (IGNORE)

In [ ]:
# FUNCTIONS IGNORE
def build_and_push_images(repo_directory:str, target_repo_names:list, target_docker_names:list, docker_username:str, push:bool = True, custom_tag_name:str = 'latest'):
    # Validate that lists are the same length
    if len(target_repo_names) != len(target_docker_names):
        raise ValueError("target_repo_names and target_docker_names must have the same length")
    
    for a_repo_name, a_docker_name in zip(target_repo_names, target_docker_names):
        repo_path = os.path.join(repo_directory, a_repo_name)
        a_docker_name_lower = a_docker_name.lower()
        docker_path = f'{docker_username}/{a_docker_name_lower}:{custom_tag_name}'
        build_cmd = ['docker', 'build','--quiet', '-f', os.path.join(repo_path, "Dockerfile"), '-t', docker_path, repo_path]
        try:
            sp.run(build_cmd)
        except Exception as e:
            raise RuntimeError(
                f"Docker build failed...\n"
                f"Build Command: {build_cmd}\n"
                f"Error: {e}"
            )
        if push:
            try:
                push_cmd = ['docker', 'push','--quiet', docker_path]
                sp.run(push_cmd)
            except Exception as e:
                raise RuntimeError(
                    f"Docker push failed...\n"
                    f"Push Command: {push_cmd}\n"
                    f"Error: {e}"
                )
            
def build_sifs_and_create_slurm_scripts(run_list, included_modules, base_dir, docker_username, build):

    for run in run_list:
        
        # Has to exist with 'mnt' structure (Doit exister avec la structure 'mnt')
        mnt_dir = os.path.join(base_dir, f'confluence_{run}', f'{run}_mnt')
        
        # Create the sh_scripts directory (Cree le repertoire sh_scripts)
        sh_dir = os.path.join(base_dir, f'confluence_{run}', 'sh_scripts')
        if not os.path.exists(sh_dir):
            os.makedirs(sh_dir)
        
        # Create the sif directory (Cree la repertoire sif)
        sif_dir = os.path.join(base_dir, f'confluence_{run}', 'sif')
        if not os.path.exists(sif_dir):
            os.makedirs(sif_dir)
        
        # Create the report directory (Cree la repertoire report)
        report_dir = os.path.join(base_dir, f'confluence_{run}', 'report')
        if not os.path.exists(report_dir):
            os.makedirs(report_dir)


        submission_prefix = '#SBATCH'


        job_details = {
        'partition': 'cpu-preempt',
        'nodes' : '1',
        'cpus-per-task': '1',
        'job-name': f'{run}_cfl',
        }
        


        command_dict = {
            'expanded_setfinder': 'singularity run --bind ' + f'{mnt_dir}/input:/data ' + os.path.join(sif_dir, 'setfinder.simg') + ' -r reaches_of_interest.json -c continent.json -e -s 17b -o /data -n /data -a MetroMan HiVDI SIC NeoBAM -i ${SLURM_ARRAY_TASK_ID}',
            'expanded_combine_data': 'singularity run --bind ' + f'{mnt_dir}/input:/data ' + os.path.join(sif_dir, 'combine_data.simg') + ' -d /data  -e -s 17b',
            'input': 'GLOBAL_INDEX=$(( ${OFFSET:-0} + SLURM_ARRAY_TASK_ID ))\n\nsingularity run --bind ' + f'{mnt_dir}/input:/mnt/data ' + os.path.join(sif_dir, 'input.simg') + ' -v 17b -t &start_time=2020-09-01T00:00:00Z&end_time=2026-01-11T00:00:00Z& -r /mnt/data/expanded_reaches_of_interest.json -i ${GLOBAL_INDEX}',
            'non_expanded_setfinder': 'singularity run --bind ' + f'{mnt_dir}/input:/data ' + os.path.join(sif_dir, 'setfinder.simg') + ' -c continent.json -s 17b -o /data -n /data -a MetroMan HiVDI SIC NeoBAM -i ${SLURM_ARRAY_TASK_ID}',
            'non_expanded_combine_data': 'singularity run --bind ' + f'{mnt_dir}/input:/data ' + os.path.join(sif_dir, 'combine_data.simg') + ' -d /data -s 17b',
            'prediagnostics': 'GLOBAL_INDEX=$(( ${OFFSET:-0} + SLURM_ARRAY_TASK_ID ))\n\nsingularity run --bind ' + f'{mnt_dir}/input:/mnt/data/input,{mnt_dir}/diagnostics/prediagnostics:/mnt/data/output ' + os.path.join(sif_dir, f'prediagnostics.simg') + ' -i ${GLOBAL_INDEX} -r reaches.json',
            'constrained_priors': f'singularity run -c --writable-tmpfs --bind {mnt_dir}/input:/mnt/data {os.path.join(sif_dir, "priors.simg")} ' + ' -i ${SLURM_ARRAY_TASK_ID} -r constrained -p usgs riggs -g -s local',
            'unconstrained_priors': f'singularity run -c --writable-tmpfs --bind {mnt_dir}/input:/mnt/data {os.path.join(sif_dir, "priors.simg")} ' + ' -i ${SLURM_ARRAY_TASK_ID} -r unconstrained -p usgs riggs -g -s local',
            'ml_priors': f'singularity run -c --writable-tmpfs --bind {mnt_dir}/input:/mnt/data {os.path.join(sif_dir, "priors.simg")} ' + ' -i ${SLURM_ARRAY_TASK_ID} -r ml -p usgs riggs -g -s local',
            'hivdi': f'singularity run --bind {mnt_dir}/input:/mnt/data/input,{mnt_dir}/flpe/hivdi:/mnt/data/output ' + os.path.join(sif_dir, 'hivdi.simg') + ' /mnt/data/input/reaches.json --input-dir /mnt/data/input -i ${SLURM_ARRAY_TASK_ID}',
            'sic4dvar': f'GLOBAL_INDEX=$(( ${{OFFSET:-0}} + SLURM_ARRAY_TASK_ID ))\n\nsingularity run --bind {mnt_dir}/input:/mnt/data/input,{mnt_dir}/flpe/sic4dvar:/mnt/data/output,{mnt_dir}/logs:/mnt/data/logs '+ os.path.join(sif_dir, 'sic4dvar.simg') + ' -r reaches.json --index ${GLOBAL_INDEX}',
            'metroman': f'GLOBAL_INDEX=$(( ${{OFFSET:-0}} + SLURM_ARRAY_TASK_ID ))\n\nsingularity run --env AWS_BATCH_JOB_ID="foo" --bind {mnt_dir}/input:/mnt/data/input,{mnt_dir}/flpe/metroman:/mnt/data/output ' + os.path.join(sif_dir, "metroman.simg") + ' -i ${GLOBAL_INDEX} -r metrosets.json -s local -v',
            'metroman_consolidation': f'GLOBAL_INDEX=$(( ${{OFFSET:-0}} + SLURM_ARRAY_TASK_ID ))\n\nsingularity run --bind {mnt_dir}/input:/mnt/data/input,{mnt_dir}/flpe/metroman:/mnt/data/flpe ' + os.path.join(sif_dir, 'metroman_consolidation.simg') + ' -i ${GLOBAL_INDEX}',
            'unconstrained_momma': f'GLOBAL_INDEX=$(( ${{OFFSET:-0}} + SLURM_ARRAY_TASK_ID ))\n\nsingularity run --bind {mnt_dir}/input:/mnt/data/input,{mnt_dir}/flpe/momma:/mnt/data/output ' + os.path.join(sif_dir, 'momma.simg') + ' -r reaches.json -m 3 -i ${GLOBAL_INDEX}',
            'constrained_momma': f'GLOBAL_INDEX=$(( ${{OFFSET:-0}} + SLURM_ARRAY_TASK_ID ))\n\nsingularity run --bind {mnt_dir}/input:/mnt/data/input,{mnt_dir}/flpe/momma:/mnt/data/output ' + os.path.join(sif_dir, 'momma.simg') + ' -r reaches.json -m 3 -c -i ${GLOBAL_INDEX}',
            'neobam': f'GLOBAL_INDEX=$(( ${{OFFSET:-0}} + SLURM_ARRAY_TASK_ID ))\n\nsingularity run --bind {mnt_dir}/input:/mnt/data/input,{mnt_dir}/flpe/geobam:/mnt/data/output ' + os.path.join(sif_dir, 'neobam.simg') + ' -r reaches.json -i ${GLOBAL_INDEX}',
            'sad': f'GLOBAL_INDEX=$(( ${{OFFSET:-0}} + SLURM_ARRAY_TASK_ID ))\n\nsingularity run --bind {mnt_dir}/input:/mnt/data/input,{mnt_dir}/flpe/sad:/mnt/data/output ' + os.path.join(sif_dir, 'sad.simg') + ' --reachfile reaches.json --index ${GLOBAL_INDEX}',
            'moi': f'GLOBAL_INDEX=$(( ${{OFFSET:-0}} + SLURM_ARRAY_TASK_ID ))\n\nsingularity run --env AWS_BATCH_JOB_ID="foo" --bind {mnt_dir}/input:/mnt/data/input,{mnt_dir}/flpe:/mnt/data/flpe,{mnt_dir}/moi:/mnt/data/output ' + os.path.join(sif_dir, 'moi.simg') + ' -j basin.json -v -b unconstrained -i ${GLOBAL_INDEX}', # -s local
            'consensus': f'GLOBAL_INDEX=$(( ${{OFFSET:-0}} + SLURM_ARRAY_TASK_ID ))\n\nsingularity run --bind {mnt_dir}/input:/mnt/data/input,{mnt_dir}/flpe:/mnt/data/flpe ' + os.path.join(sif_dir, 'consensus.simg') + ' --mntdir /mnt/data -r /mnt/data/input/reaches.json -i ${GLOBAL_INDEX}',
            'unconstrained_offline': f'GLOBAL_INDEX=$(( ${{OFFSET:-0}} + SLURM_ARRAY_TASK_ID ))\n\nsingularity run --bind {mnt_dir}/input:/mnt/data/input,{mnt_dir}/flpe:/mnt/data/flpe,{mnt_dir}/moi:/mnt/data/moi,{mnt_dir}/offline:/mnt/data/output ' + os.path.join(sif_dir, 'offline.simg') + ' unconstrained timeseries integrator reaches.json ${GLOBAL_INDEX}',
            'validation': f'GLOBAL_INDEX=$(( ${{OFFSET:-0}} + SLURM_ARRAY_TASK_ID ))\n\nsingularity run --bind {mnt_dir}/input:/mnt/data/input,{mnt_dir}/flpe:/mnt/data/flpe,{mnt_dir}/moi:/mnt/data/moi,{mnt_dir}/offline:/mnt/data/offline,{mnt_dir}/validation:/mnt/data/output ' + os.path.join(sif_dir, 'validation.simg') + ' -r reaches.json -t unconstrained -i ${GLOBAL_INDEX}',
            'output': f'singularity run --bind {mnt_dir}/input:/mnt/data/input,{mnt_dir}/flpe:/mnt/data/flpe,{mnt_dir}/diagnostics:/mnt/data/diagnostics,{mnt_dir}/moi:/mnt/data/moi,{mnt_dir}/offline:/mnt/data/offline,{mnt_dir}/validation:/mnt/data/validation,{mnt_dir}/output:/mnt/data/output ' + os.path.join(sif_dir, 'output.simg') + ' -s local -j /app/metadata/metadata.json -m input priors prediagnostics momma hivdi neobam metroman sic4dvar sad validation swot -i ${SLURM_ARRAY_TASK_ID}'
        }
        
 

        def create_slurm_script(job_details=job_details, build_image=False, sif_dir='foo'):
            submission_prefix = job_details['submission_prefix']
            if build_image:
                module_name = job_details['module_name']
                image_name = module_name.replace('expanded_', '').replace('non_', '').replace('unconstrained_', '').replace('constrained_', '')
                sp.run(['singularity', 'build', '-F', os.path.join(sif_dir, image_name + '.simg'), f"docker://{job_details['docker_username']}/{image_name}"])

            file = open(os.path.join(sh_dir, f'{module_to_run}.sh'), 'w')
            file.write('#!/bin/bash \n')
            file.write(f'{submission_prefix} -o {os.path.join(report_dir, f"{module_to_run}.%a.out")}' + ' \n')

            for item in job_details:
                if item not in ['run_command', 'module_name', 'docker_username', 'submission_prefix']:
                    file.write(f'{submission_prefix} --{item}={job_details[item]} \n')
            file.write(job_details["run_command"])
            file.close()

        included_modules = included_modules

        for module_to_run, run_command in command_dict.items():
            
            if module_to_run == 'moi':
                time_to_use = '00:30:00'
                mem_to_use = '2G'
            elif module_to_run == 'neobam':
                time_to_use = '00:30:00'
                mem_to_use = '48G'
            elif module_to_run == 'output':
                time_to_use = '05:00:00'
                mem_to_use = '2G'
            else:
                time_to_use = '02:00:00'
                mem_to_use = '4G'
                
            if included_modules:
                if module_to_run not in included_modules:
                    continue

            print('DIRECTORY NAME: ', run, '\nMODULE: ', module_to_run)
            


            job_details.update({
                'run_command': run_command,
                'module_name': module_to_run,
                'mem': mem_to_use,
                'time': time_to_use,
                'docker_username': docker_username,
                'submission_prefix': submission_prefix,
                'job-name': f'{module_to_run}_{run}_cfl',

            })
            create_slurm_script(job_details=job_details, build_image=build, sif_dir=sif_dir)

                
def generate_slurm_driver(
    job_name: str,
    output_log_dir: str,
    partition: str,
    time_limit: str,
    nodes: int,
    ntasks: int,
    cpus_per_task: int,
    mem: str,
    run: str,
    directory: str,
    json_file: str,
    batch_size: int,
    concurrent_jobs: int,
    script_jobs: dict[str, str],
    scripts: list[str]
) -> str:
    slurm_header = f"""#!/bin/bash
#SBATCH --job-name={job_name}
#SBATCH --output={output_log_dir}/{job_name}_%j.out
#SBATCH --error={output_log_dir}/{job_name}_%j.err
#SBATCH --partition={partition}
#SBATCH --time={time_limit}
#SBATCH --nodes={nodes}
#SBATCH --ntasks={ntasks}
#SBATCH --cpus-per-task={cpus_per_task}
#SBATCH --mem={mem}

run='{run}'
echo "Run: $run"

directory="{directory}"

# Parameters
json_file="{json_file}"
expanded_json_file="{expanded_json_file}"
reach_json_file="{reach_json_file}"
basin_json_file="{basin_json_file}"
metroman_json_file="{metroman_json_file}"
default_jobs=$(jq length "$json_file")

# Adjust to HPC requirements
batch_size={batch_size}
concurrent_jobs={concurrent_jobs}

# Map specific script names to their job counts
declare -A script_jobs=(
"""

    # Inject job counts into script_jobs associative array
    for script, jobs in script_jobs.items():
        slurm_header += f"    [{script}]={jobs}\n"
    slurm_header += ")\n\n"

    # Build scripts array
    script_array = '    ' + '\n    '.join(scripts)
    scripts_block = f"""scripts=(
{script_array}
)
"""

    body = f"""{scripts_block}




for slurm_script in "${{scripts[@]}}"; do
    echo "Starting submission for: $slurm_script"
    date


    # Change Job Numbers
    if [[ -s "$expanded_json_file" ]]; then
      expended_jobs=$(jq length "$expanded_json_file")
      script_jobs["input.sh"]=$expended_jobs
    fi

    if [[ -s "$reach_json_file" ]]; then
      reaches_jobs=$(jq length "$reach_json_file")
      script_jobs["prediagnostics.sh"]=$reaches_jobs
      script_jobs["sad.sh"]=$reaches_jobs
      script_jobs["sic4dvar.sh"]=$reaches_jobs
      script_jobs["unconstrained_momma.sh"]=$reaches_jobs
      script_jobs["constrained_momma.sh"]=$reaches_jobs
      script_jobs["neobam.sh"]=$reaches_jobs
      script_jobs["consensus.sh"]=$reaches_jobs
      script_jobs["unconstrained_offline.sh"]=$reaches_jobs
      script_jobs["validation.sh"]=$reaches_jobs
    fi

    if [[ -s "$basin_json_file" ]]; then
      basin_jobs=$(jq length "$basin_json_file")
      script_jobs["moi.sh"]=$basin_jobs
    fi

    if [[ -s "$metroman_json_file" ]]; then
      metroman_jobs=$(jq length "$metroman_json_file")
      script_jobs["metroman.sh"]=$metroman_jobs
      script_jobs["metroman_consolidation.sh"]=$metroman_jobs
    fi


    num_jobs="${{script_jobs[$slurm_script]}}"
    # echo "Current job counts:"
    # declare -p script_jobs
    
    if [[ -z "$num_jobs" ]]; then
        echo "Warning: No job count found for $slurm_script. Skipping."
        continue
    fi

    start=0
    while [ $start -lt $num_jobs ]; do
        end=$((start + batch_size - 1))
        if [ $end -ge $num_jobs ]; then
            end=$((num_jobs - 1))
        fi

        echo "Submitting jobs $start to $end from $slurm_script"
        job_id=$(sbatch --export=ALL,OFFSET=${{start}} --array=0-$((end - start))%${{concurrent_jobs}} "${{directory}}/${{slurm_script}}")
        # job_id=$(sbatch --array=${{start}}-${{end}}%${{concurrent_jobs}} "${{directory}}/${{slurm_script}}")
        job_id_number=$(echo $job_id | awk '{{print $4}}')

        echo "Waiting for job array $job_id_number to finish..."
        while squeue -j "$job_id_number" 2>/dev/null | grep -q "$job_id_number"; do
            job_info=$(squeue -j "${{job_id_number}}[]" --noheader -o "%i %T %R")
            held_tasks=$(echo "$job_info" | grep -i "launch failed requeued held" | awk '{{print $1}}')

            if [[ -n "$held_tasks" ]]; then
                echo "Detected held tasks in array $job_id_number:"
                echo "$held_tasks"
                for task in $held_tasks; do
                    echo "Cancelling task $task..."
                    scancel "$task"
                done
            fi

            sleep 10
        done

        echo "Batch $job_id_number has finished. Submitting next batch."
        date

        start=$((end + 1))
        sleep 5
    done      
    
done

echo "Run $run has finished successfully."
"""
    return slurm_header + body


---
### Prepare Docker Images (RUN LOCALLY, NOT HPC)
* Builds docker images locally and stores them on your dockerhub

In [ ]:
import os
import subprocess as sp

In [ ]:
#------------------------------------------------

# SETUP

# Directory where you are storing repos
repo_directory = '/Users/elisafriedmann/Library/CloudStorage/OneDrive-UniversityofMassachusetts/UMass/SWOT/confluence'
target_repo_names = ['setfinder', 'momma', 'offline-discharge-data-product-creation'] # names of modules to build and likely push to Docker Hub
target_docker_names = ['setfinder', 'momma', 'offline'] # lowercase required, must match command_dict

# Only provide this if you want to store images on dockerhub to move to HPC (you probably do)
push = True
docker_username = 'efrie130' # replace with your Docker Hub username
custom_tag_name = 'localTest' # leave this as latest unless you are testing multiple confluence versions

# --------------------------------------------------------------------------------------

In [ ]:
build_and_push_images(\
    repo_directory = repo_directory, \
    target_repo_names = target_repo_names, \
    target_docker_names = target_docker_names, \
    docker_username = docker_username, \
    push = push, \
    custom_tag_name = custom_tag_name \
)
                      
# The output should look something like 
# sha256:6900c3d99325a4a7c8b282d4a7a62f2a0f3fc673f03f5ca3333c2746bf20d06a
# docker.io/efrie130/validation:latest

#command line version of build_and_push_images function above - verbose
# docker build -t efrie130/validation:latest ./Validation/ && docker push efrie130/validation:latest &


---
### Confluence Module SLURM Script Generator (RUN ON HPC, NOT LOCALLY)
* Download empty /mnt structure
* Build sif files from your dockerhub and generates scripts to submit to a SLURM job scheduler

In [29]:
#-------------------------------------------------
import os
import subprocess as sp

# SETUP
## FIRST TIME:
# Install empty /mnt directory with input data and eventual output data
# Command line: gdown 1xRltFZ1gyP_nvwHMJW-rIgClzXx8CSLC


# Directory where you are storing repos
base_dir = '/Users/elisafriedmann/Library/CloudStorage/OneDrive-UniversityofMassachusetts/UMass/SWOT/confluence/run-confluence-locally/'
included_modules= {'expanded_combine_data', 'expanded_setfinder', 'input'}
docker_username = 'efrie130'
custom_tag_name = 'latest' # leave this as latest unless you have a really good reason!

# Providing a run list will create slurm scripts to run 
run_list = ['run_1'] #one name per mnt

# Rebuild the sif
build = True

#-------------------------------------------------


In [30]:
build_sifs_and_create_slurm_scripts(run_list=run_list, \
                                    included_modules = included_modules, \
                                    base_dir = base_dir, \
                                    docker_username = docker_username,
                                    build = build
                                   )

DIRECTORY NAME:  run_1 
MODULE:  expanded_setfinder


FileNotFoundError: [Errno 2] No such file or directory: 'singularity'

---
### Confluence Driver Script Generator (RUN ON HPC, NOT LOCALLY)
* Creates a batch submission script that will run all of your sif files in serial
* use sbatch to submit the entire run
* low resources and a long time should be used here, as all this job will do is launch your SLURM scripts you created for each module, it is basically a job manager

In [ ]:
# Create driver SLURM script for each run in run_list

for run in run_list:

    job_name = str(run)
    output_log_dir = f"/nas/cee-water/cjgleason/heejin/SWOT_Confluence/confluence_run/confluence_{run}/log"
    partition = "cpu-preempt"
    time_limit = "30:00:00"
    nodes = 1
    ntasks = 1
    cpus_per_task = 1
    mem = "40G"

    run = str(run)
    directory = f"/nas/cee-water/cjgleason/heejin/SWOT_Confluence/confluence_run/confluence_{run}"
    sh_directory = f"{directory}/sh_scripts"
    json_file = f"{directory}/{run}_mnt/input/reaches_of_interest.json"
    expanded_json_file = f"{directory}/{run}_mnt/input/expanded_reaches_of_interest.json"
    reach_json_file = f"{directory}/{run}_mnt/input/reaches.json"
    basin_json_file = f"{directory}/{run}_mnt/input/basin.json"
    metroman_json_file = f"{directory}/{run}_mnt/input/metrosets.json"
    

    batch_size = 1000
    concurrent_jobs = 400

    # modify as needed for modules you need to run
    script_jobs = {
        "expanded_setfinder.sh": "7",
        "expanded_combine_data.sh": "1",
        "input.sh": "$default_jobs",
        "non_expanded_setfinder.sh": "7",
        "non_expanded_combine_data.sh": "1",
        # "unconstrained_priors.sh": "7", 
        # "sad.sh": "$default_jobs",
        # "prediagnostics.sh": "$default_jobs",
        # "metroman.sh": "$default_jobs",
        # "metroman_consolidation.sh": "$default_jobs",
        # "sic4dvar.sh": "$default_jobs",
        # "unconstrained_momma.sh": "$default_jobs",
        # "constrained_momma.sh": "$default_jobs",
        # "neobam.sh": "$default_jobs",
        # "moi.sh": "$default_jobs",
        # "consensus.sh": "$default_jobs",
        # "unconstrained_offline.sh": "$default_jobs",
        # "validation.sh": "$default_jobs",
        # "output.sh": "7",
    }

    scripts = [
        "expanded_setfinder.sh",
        "expanded_combine_data.sh",
        "input.sh",
        "non_expanded_setfinder.sh",
        "non_expanded_combine_data.sh",
        # "unconstrained_priors.sh",
        # "sad.sh",
        # "prediagnostics.sh",
        # "metroman.sh",
        # "metroman_consolidation.sh",
        # "sic4dvar.sh",
        # "unconstrained_momma.sh",
        # "constrained_momma.sh",
        # "neobam.sh",
        # "moi.sh",
        # "consensus.sh",
        # "unconstrained_offline.sh",
        # "validation.sh",
        # "output.sh",
    ]

    driver_script = generate_slurm_driver(
        job_name=job_name,
        output_log_dir=output_log_dir,
        partition=partition,
        time_limit=time_limit,
        nodes=nodes,
        ntasks=ntasks,
        cpus_per_task=cpus_per_task,
        mem=mem,
        run=run,
        directory=sh_directory,
        json_file=json_file,
        batch_size=batch_size,
        concurrent_jobs=concurrent_jobs,
        script_jobs=script_jobs,
        scripts=scripts,
    )

    # Save to file
    with open(f"/nas/cee-water/cjgleason/heejin/SWOT_Confluence/confluence_run/confluence_{run}/slurm_driver.sh", "w") as f:
        f.write(driver_script)


# Optionally submit
# import subprocess
# subprocess.run(["sbatch", "driver_submit.sh"], check=True)


---
# Running Tests

#### In order to run on specific reaches
* modify the file at /mnt/input/reaches_of_interest.json
#### In order to change a module and test it
### Option 1
* change the module locally, build it and push to dockerhub using the first part of this notebook and then run as usual
* you can use the run_list variable to generate more submission script per moule to test more than one change at a time. However, whenver you submit them, they will still run one at a time, it just submits the next run automatically.

### Option 2
* Use a symlink to connect a previous run to a new directory
* Run module of interest using the data in previous modules (only need to run the changed module!)